In [1]:
# ! git clone https://github.com/ClaudioPaterniti/MinesweeperLearning.git

In [ ]:
# import os
# os.chdir('/content/MinesweeperLearning')

In [3]:
import json

import numpy as np
import matplotlib.pyplot as plt
import torch

from src.game import Game
from src.models.patch_mlp import PatchMLPModel
from src.models.unet import UnetModel
from src.models.conv import ConvModel
from src.player import *

In [4]:

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(device)

cuda


In [5]:
result_file = 'weights/scores.json'
r, c, m, n = (16, 30, 99, 50_000)

In [6]:
models: list[tuple[MinesweeperModel, str]] = [
    (ConvModel, 'weights/conv_3x3_64.pth'),
    (ConvModel, 'weights/conv_3x3_128.pth'),
    (ConvModel, 'weights/conv_5x5_64.pth'),
    (ConvModel, 'weights/conv_5x5_128.pth'),
    (ConvModel, 'weights/conv_7x7_64.pth'),
    (PatchMLPModel, 'weights/patch_mlp_7x7_512.pth'),
    (PatchMLPModel, 'weights/patch_mlp_7x7_1024.pth'),
    (UnetModel, 'weights/unet_16x30_64.pth'),
]

In [7]:
games = Game(r, c, m, n)
zeros = games.open_zero()
win_rates = {}
max_win_rate = 0

In [8]:
for model_class, path in models:
    model = model_class.load(path, device)
    player = ThresholdPlayer(model)
    games.reset()
    games.move(zeros)
    player.play(games)
    win_rate = games.win_rate()
    win_rates[path] = win_rate
    print(f'{path} win rate: {win_rate}')
    if win_rate > max_win_rate:
        max_win_rate = win_rate
        best_model = model

weights/conv_3x3_64.pth win rate: 0.44852
weights/conv_3x3_128.pth win rate: 0.4426
weights/conv_5x5_64.pth win rate: 0.43574
weights/conv_5x5_128.pth win rate: 0.41632
weights/conv_7x7_64.pth win rate: 0.43528
weights/patch_mlp_7x7_512.pth win rate: 0.30264
weights/patch_mlp_7x7_1024.pth win rate: 0.31012
weights/unet_16x30_64.pth win rate: 0.40506


In [9]:
with open(result_file, 'w+') as fp:
    json.dump({'n': n, 'winRates': win_rates}, fp, indent=4)

In [10]:
for thresholds in [(0.05, 0.95), (0.02, 0.98), (0.015, 0.985), (0.01, 0.99)]:
    player = ThresholdPlayer(best_model, *thresholds)
    games.reset()
    games.move(zeros)
    player.play(games)
    win_rate = games.win_rate()
    print(f'{thresholds} win rate: {win_rate}')

(0.05, 0.95) win rate: 0.3831
(0.02, 0.98) win rate: 0.43894
(0.015, 0.985) win rate: 0.44418
(0.01, 0.99) win rate: 0.44852
